In [1]:
# MRI STUFF
# Test increase in layers 
# Increase in layer architectures
# Increase in drop out 
# With class weights, without class weights 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import (classification_report, confusion_matrix, balanced_accuracy_score, 
accuracy_score, roc_auc_score, auc)
from sklearn.model_selection import KFold, StratifiedKFold
import plotly.graph_objects as go
import glob


In [3]:
#drive.mount('/content/drive')

#ROOT_DIR = "/content/drive/MyDrive/Spring 2022/DTSC 870/Code" # HC Directory
#ROOT_DIR = "/content/drive/MyDrive/DTSC 870/Code" #MT Directory
KAGGLE_START = "/kaggle/input"
ROOT_DIR = KAGGLE_START
DATASET_01_TEST = KAGGLE_START + "/mri-test-set/test"
DATASET_01_TRAIN = KAGGLE_START + "/mri-trainset/train"
KAGGLE_OUT = "/kaggle/working"



In [4]:
def transfer_learning_model_dense169_1():

  IMG_SHAPE = (128,128,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  x = tf.keras.applications.densenet.preprocess_input(inputs)
  base_model = tf.keras.applications.densenet.DenseNet169(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 500
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  prediction_layer = tf.keras.layers.Dense(1)
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_dense169 = transfer_learning_model_dense169_1()
transfer_model_dense169.summary()

2022-05-10 17:18:01.084726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 17:18:01.179600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 17:18:01.180400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 17:18:01.181558: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

51888128/51877672 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 128, 128, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 128, 128, 3)       0         
_________________________________________________________________
densenet169 (Functional)     (None, 4, 4, 1664)        12642880  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1664)              0         
______________________________________________________________

In [5]:
datagen = ImageDataGenerator()
image_size = (256, 256)
test_set = datagen.flow_from_directory(DATASET_01_TEST, target_size=image_size, batch_size=32, class_mode='binary', shuffle=False)
train_set = datagen.flow_from_directory(DATASET_01_TRAIN, target_size=image_size, batch_size=32, class_mode='binary')


Found 121 images belonging to 2 classes.
Found 279 images belonging to 2 classes.


In [6]:
def create_model_V1():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model


In [7]:
def create_model_V1_02():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [8]:
def create_model_V1_04():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [9]:
def create_model_V2():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [10]:
def create_model_V2_02():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [11]:
def create_model_V2_04():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [12]:
def create_model_V3():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [13]:
def create_model_V3_02():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [14]:
def create_model_V3_04():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [15]:
def create_model_V4():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [16]:
def create_model_V4_02():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [17]:
def create_model_V4_04():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.4))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [18]:
#modify this 
norm_weight = 400/(2*(118+52))
tumor_weight = 400/(2*(161+69))

class_weights = {0:norm_weight, 1:tumor_weight}

In [19]:
import glob

#borrowed from svm code
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-4] + "/" + img.split("/")[-3] + "/" + img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [20]:
test_train = get_data_df(DATASET_01_TRAIN+"/*/*.jpg")
test_train.head()

,filename,label
0,mri-trainset/train/Tumor/Tumor (77).jpg,Tumor
1,mri-trainset/train/Tumor/Tumor (116).jpg,Tumor
2,mri-trainset/train/Tumor/Tumor (86).jpg,Tumor
3,mri-trainset/train/Tumor/Tumor (49).jpg,Tumor
4,mri-trainset/train/Tumor/Tumor (40).jpg,Tumor


In [21]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_01_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
idg = ImageDataGenerator(rescale=1./255)

#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size):
  skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
  train_data = get_data_df(DATASET_01_TRAIN+"/*/*.jpg")
  Y = train_data[['label']]
  n = len (train_data)
  #Incorperate preprocessing in the method 
  #####################################################################
  #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
  #aug_train['filename'] = aug_train['filename'].str[7:]
  #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


  #######################################################################
  idg = ImageDataGenerator()
  #idg = ImageDataGenerator()
  #Y = final_train[['label']]
  #n = len(final_train)    
  VALIDATION_ACCURACY = []
  VALIDATION_LOSS = []

  #create a directory
  save_dir = KAGGLE_OUT+'/datasets/01_MRI/Saved_CNN_Models/Testing/'+model_method.__name__+'/AugTest'+'/'+str(epoch)+'epochs/'
  img_dir = ROOT_DIR

  fold_var = 1

  #n is number of samples, doesn't care about x values for split, just y
  for train_index, val_index in skf.split(np.zeros(n),Y):

    #Index value for fold
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    #generator for train
    #print("hi")
    #Modify the training size, have an input adjustability 
    train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                      target_size=image_size, batch_size=32,
                    x_col = "filename", y_col = "label",
                    class_mode = "binary", shuffle = True)
    #print("hi")
    #generator for validation
    valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                target_size=image_size, batch_size=32,
                x_col = "filename", y_col = "label",
                class_mode = "binary", shuffle = True)
    #print("hi")
    #create a method for this
    # CREATE NEW MODEL
    model = model_method()
    # COMPILE NEW MODEL
    #Binary cross entropy, opt will be adam, metric will be binary accuracy?
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001),
            metrics=['binary_accuracy'])
    
    # CREATE CALLBACKS
    #Unsure about checpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                monitor='val_binary_accuracy', verbose=1, 
                save_best_only=True, mode='max')
  
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    #Include the weights for classes
    history = model.fit(train_data_generator,
            epochs=epoch,
            callbacks=callbacks_list,
            validation_data=valid_data_generator,
                       class_weight = class_weights)
  
    #PLOT HISTORY
    #Plots for whichever
    #		:
    #plt.plot(history.history['binary_accuracy'], label='accuracy')
    #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
    #plt.xlabel('Epoch')
    #plt.ylabel('Accuracy')
    #plt.ylim([0.3, 1])
    #plt.legend(loc='lower right')
    #plt.show()
    #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
    #fig = px.line(frame)
    #fig.show()
    fig = go.Figure()
    fig.add_trace(go.Scatter(y= history.history['binary_accuracy'],
                      mode='lines+markers',
                      name='Training Accuracy'))
    fig.add_trace(go.Scatter(y= history.history['val_binary_accuracy'],
                      mode='lines+markers',
                      name='Validation Accuracy'))
    fig.update_yaxes(range=[0.40, 1])
    fig.update_layout(
      title="Fold "+ str(fold_var)+ "'s performance",
      title_x=0.5,
      xaxis_title="Epochs",
      yaxis_title="Accuracy",
      autosize=False,
      width=800,
      height=400,
      margin=dict(l=20, r=20, t=40, b=20),)
    fig.show()
    #		:
    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")
    
    results = model.evaluate(test_set)
    
    results = dict(zip(model.metrics_names,results))
    
    VALIDATION_ACCURACY.append(results['binary_accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1

In [22]:
#model_test(5, 30, create_model_V1, (256,256))

In [23]:
#model_test(5, 30, create_model_V1_02, (256,256))

In [24]:
#model_test(5, 30, create_model_V1_04, (256,256))

In [25]:
#model_test(5, 30, create_model_V2, (256,256))

In [26]:
#model_test(5, 30, create_model_V2_02, (256,256))

In [27]:
#model_test(5, 30, create_model_V2_04, (256,256))

In [28]:
#model_test(5, 30, create_model_V3, (256,256))

In [29]:
#model_test(5, 30, create_model_V3_02, (256,256))

In [30]:
#model_test(5, 30, create_model_V3_04, (256,256))

In [31]:
#model_test(5, 30, create_model_V4, (256,256))

In [32]:
#model_test(5, 30, create_model_V4_02, (256,256))

In [33]:
#model_test(5, 30, create_model_V4_04, (256,256))

In [34]:
def transfer_learning_model_dense169_2():

  IMG_SHAPE = (256,256,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  x = tf.keras.applications.densenet.preprocess_input(inputs)
  base_model = tf.keras.applications.densenet.DenseNet169(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
  #print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 350
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  prediction_layer = tf.keras.layers.Dense(1)
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

In [35]:
model_test(5, 30, transfer_learning_model_dense169_2, (256,256))

Found 223 validated image filenames belonging to 2 classes.
Found 56 validated image filenames belonging to 2 classes.


2022-05-10 17:18:16.457186: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-05-10 17:18:27.736748: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


7/7 [==============================] - 23s 884ms/step - loss: 4.5218 - binary_accuracy: 0.5561 - val_loss: 7.7946 - val_binary_accuracy: 0.4464

Epoch 00001: val_binary_accuracy improved from -inf to 0.44643, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_1.h5
Epoch 2/30
7/7 [==============================] - 2s 243ms/step - loss: 2.2112 - binary_accuracy: 0.7848 - val_loss: 7.7419 - val_binary_accuracy: 0.4821

Epoch 00002: val_binary_accuracy improved from 0.44643 to 0.48214, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_1.h5
Epoch 3/30
7/7 [==============================] - 1s 176ms/step - loss: 1.1201 - binary_accuracy: 0.8744 - val_loss: 7.5630 - val_binary_accuracy: 0.4821

Epoch 00003: val_binary_accuracy did not improve from 0.48214
Epoch 4/30

4/4 [==============================] - 1s 365ms/step - loss: 0.2519 - binary_accuracy: 0.9256
Found 223 validated image filenames belonging to 2 classes.
Found 56 validated image filenames belonging to 2 classes.
Epoch 1/30
7/7 [==============================] - 14s 652ms/step - loss: 4.3084 - binary_accuracy: 0.5650 - val_loss: 4.0080 - val_binary_accuracy: 0.6786

Epoch 00001: val_binary_accuracy improved from -inf to 0.67857, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_2.h5
Epoch 2/30
7/7 [==============================] - 1s 171ms/step - loss: 2.8224 - binary_accuracy: 0.7668 - val_loss: 6.7706 - val_binary_accuracy: 0.4821

Epoch 00002: val_binary_accuracy did not improve from 0.67857
Epoch 3/30
7/7 [==============================] - 1s 178ms/step - loss: 2.3012 - binary_accuracy: 0.8206 - val_loss: 7.0348 - val_binary_accuracy: 0.4821

Epoch 00003: val_binary_

4/4 [==============================] - 1s 107ms/step - loss: 2.4623 - binary_accuracy: 0.8182
Found 223 validated image filenames belonging to 2 classes.
Found 56 validated image filenames belonging to 2 classes.
Epoch 1/30
7/7 [==============================] - 14s 568ms/step - loss: 2.7821 - binary_accuracy: 0.7040 - val_loss: 6.4699 - val_binary_accuracy: 0.5000

Epoch 00001: val_binary_accuracy improved from -inf to 0.50000, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_3.h5
Epoch 2/30
7/7 [==============================] - 1s 175ms/step - loss: 1.4935 - binary_accuracy: 0.8789 - val_loss: 3.5575 - val_binary_accuracy: 0.6071

Epoch 00002: val_binary_accuracy improved from 0.50000 to 0.60714, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_3.h5
Epo

4/4 [==============================] - 1s 95ms/step - loss: 0.7768 - binary_accuracy: 0.9339
Found 223 validated image filenames belonging to 2 classes.
Found 56 validated image filenames belonging to 2 classes.
Epoch 1/30
7/7 [==============================] - 15s 583ms/step - loss: 2.3866 - binary_accuracy: 0.7444 - val_loss: 7.6311 - val_binary_accuracy: 0.4464

Epoch 00001: val_binary_accuracy improved from -inf to 0.44643, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_4.h5
Epoch 2/30
7/7 [==============================] - 1s 182ms/step - loss: 1.3999 - binary_accuracy: 0.8655 - val_loss: 8.2634 - val_binary_accuracy: 0.4643

Epoch 00002: val_binary_accuracy improved from 0.44643 to 0.46429, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_4.h5
Epoc

4/4 [==============================] - 1s 115ms/step - loss: 0.6804 - binary_accuracy: 0.9256
Found 224 validated image filenames belonging to 2 classes.
Found 55 validated image filenames belonging to 2 classes.
Epoch 1/30
7/7 [==============================] - 15s 677ms/step - loss: 7.7244 - binary_accuracy: 0.4241 - val_loss: 8.2530 - val_binary_accuracy: 0.4182

Epoch 00001: val_binary_accuracy improved from -inf to 0.41818, saving model to /kaggle/working/datasets/01_MRI/Saved_CNN_Models/Testing/transfer_learning_model_dense169_2/AugTest/30epochs/transfer_learning_model_dense169_2_model_5.h5
Epoch 2/30
7/7 [==============================] - 1s 177ms/step - loss: 7.7244 - binary_accuracy: 0.4241 - val_loss: 8.2620 - val_binary_accuracy: 0.4182

Epoch 00002: val_binary_accuracy did not improve from 0.41818
Epoch 3/30
7/7 [==============================] - 1s 178ms/step - loss: 7.7244 - binary_accuracy: 0.4241 - val_loss: 8.2789 - val_binary_accuracy: 0.4182

Epoch 00003: val_binary_

4/4 [==============================] - 1s 112ms/step - loss: 5.9413 - binary_accuracy: 0.5785
